In [1]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import shapely
from shapely import Point, box
import xagg as xa
import rioxarray

# supress warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

Found esmf.mk at: /home/malasawedah/.conda/envs/test2/lib/esmf.mk


In [2]:
aoi = gpd.read_file('/mnt/CEPH_PROJECTS/ScaleAgData/03_Ancillary_Data/LAFIS_zonal_stats_prato_stabile_1000.shp')
grid5 = pd.read_csv('/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/polygons_grid_5')
s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc')

In [3]:
s1 = s1.fillna(0)

In [4]:
s1.rio.write_crs("EPSG:32632", inplace=True)

<xarray.Dataset>
Dimensions:  (t: 12, y: 2282, x: 3764)
Coordinates:
    crs      int64 0
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    VV       (t, y, x) float32 0.0 0.0 0.0 0.0 ... 0.06706 0.1505 0.06671 0.1619
    VH       (t, y, x) float32 0.0 0.0 0.0 0.0 ... 0.04054 0.0371 0.02428
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [5]:
s1_wgs84 = s1.rio.reproject("EPSG:4326")

In [6]:
grid5['geometry'] = grid5['geometry'].apply(lambda x: shapely.wkt.loads(x))
gdf_grid5 = gpd.GeoDataFrame(grid5, geometry='geometry')
gdf_grid5.crs = 'EPSG:4326'
gdf_grid5.head(3)

,Unnamed: 0,ID,FID,CUAA,CODE,DESCR_IT,DESCR_DE,area,dem_mean,dem_median,slo_mean,slo_median,asp_mean,asp_median,north_mean,north_medi,dem_class,geometry
0,0,2894152,PRGMFR75E13F132Z-1688030,PRGMFR75E13F132Z,AP2,Prato stabile,Wiese (Dauerwiese),56,1301.867221,1301.705200,17.897289,17.806307,85.131834,84.665092,1.000000,1.0,1300.0,"POLYGON ((11.02662 46.46839, 11.02669 46.46830..."
1,1,2347633,RNRGTH80H18F132V-59,RNRGTH80H18F132V,AP2,Prato stabile,Wiese (Dauerwiese),31147,1302.440111,1302.553589,22.939459,22.398680,145.304115,153.931000,0.043887,0.0,1300.0,"POLYGON ((10.94554 46.65611, 10.94556 46.65604..."
2,2,2970168,WSSCRS85L19A952F-2158250,WSSCRS85L19A952F,AP2,Prato stabile,Wiese (Dauerwiese),842,1318.083754,1317.940063,12.269650,12.262682,228.961556,228.782425,0.000000,0.0,1300.0,"POLYGON ((11.13220 46.49439, 11.13249 46.49438..."


In [7]:
%%time
weightmap = xa.pixel_overlaps(s1_wgs84, gdf_grid5)

creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...
success!
CPU times: user 2min 25s, sys: 4.07 s, total: 2min 29s
Wall time: 2min 29s


In [ ]:
ds = s1_wgs84
wm = weightmap
impl='for_loop'
stat=['mean']
skipna=True
interpolate_NaN=False
silent=False

In [10]:
aggregated = xa.aggregate(s1_wgs84,weightmap,stat=['mean','min','max'])
#['mean']

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating VV by mean, min, max...
aggregating VH by mean, min, max...
all variables aggregated to polygons!


In [11]:
aggregated.to_dataset()

<xarray.Dataset>
Dimensions:     (poly_idx: 101, t: 12)
Coordinates:
  * poly_idx    (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * t           (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-03
Data variables: (12/23)
    Unnamed: 0  (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
    ID          (poly_idx) int64 2894152 2347633 2970168 ... 2060657 2614883
    FID         (poly_idx) object 'PRGMFR75E13F132Z-1688030' ... 'NTRSFR65C25...
    CUAA        (poly_idx) object 'PRGMFR75E13F132Z' ... 'NTRSFR65C25I431Z'
    CODE        (poly_idx) object 'AP2' 'AP2' 'AP2' 'AP2' ... 'AP2' 'AP2' 'AP2'
    DESCR_IT    (poly_idx) object 'Prato stabile' ... 'Prato stabile'
    ...          ...
    VV_mean     (poly_idx, t) float64 nan nan nan nan ... 0.0 0.04976 0.0 0.0554
    VH_mean     (poly_idx, t) float64 nan nan nan nan ... 0.01559 0.0 0.01323
    VV_min      (poly_idx, t) float64 nan nan nan nan ... 0.005941 0.0 0.005757
    VV_max      (poly_idx, t) float64 nan nan nan nan ... 0.0 0.2604 0.0 0.3252
    VH_min      (poly_idx, t) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    VH_max      (poly_idx, t) float64 nan nan nan nan ... 0.0 0.0653 0.0 0.06992

In [16]:
agg_obj = aggregated

In [17]:
ds_out = xr.Dataset(coords={'poly_idx':(['poly_idx'],agg_obj.agg.poly_idx.values)})

In [18]:
ds_out

<xarray.Dataset>
Dimensions:   (poly_idx: 101)
Coordinates:
  * poly_idx  (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99 100
Data variables:
    *empty*

In [12]:
SUPPORTED_STATISTICS_METHODS = ["weighted_mean", "weighted_sum","weighted_median", "mean", "max", "min", "sum", "median", "count", "std"]


In [32]:
from functools import reduce


In [ ]:
"""
# Add other polygon attributes
for var in [c for c in agg_obj.agg.columns if c not in ['poly_idx','rel_area','poly_area', 'pix_idxs','coords']]:    
    if (var not in agg_obj.ds_in.var()) and (not any(var_stat in var for var_stat in SUPPORTED_STATISTICS_METHODS) ):

        # For auxiliary variables (from the shapefile), just copy them wholesale into the dataset
        ds_out[var] = xr.DataArray(data=agg_obj.agg[var],coords=[agg_obj.agg.poly_idx],dims=['poly_idx'])
    else:
        original_var = reduce(lambda s, sub: s.replace(sub, ''), SUPPORTED_STATISTICS_METHODS, var)
        original_var = original_var.replace('_', '')
        # For data variables (from the input grid), create empty array
        ds_out[var] = xr.DataArray(data=np.zeros((len(agg_obj.agg),
                                                     *[agg_obj.ds_in[original_var].sizes[k] for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]))*np.nan,
                                   dims=['poly_idx',*[k for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]],
                                   coords=[[k for k in agg_obj.agg.poly_idx],*[agg_obj.ds_in[original_var][k].values for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]])

        # Now insert aggregated values 
        for poly_idx in agg_obj.agg.poly_idx:
            ds_out[var].loc[{'poly_idx':poly_idx}] = np.squeeze(agg_obj.agg.loc[poly_idx,var])


# Add non-geographic coordinates for the variables to be aggregated
for crd in [k for k in agg_obj.ds_in.sizes.keys() if (k not in ['lat','lon','loc','bnds'])]:
    ds_out[crd] = xr.DataArray(dims=[crd],
                               data=agg_obj.ds_in[crd].values,
                               coords=[agg_obj.ds_in[crd].values])
                               """

In [33]:
for var in [c for c in agg_obj.agg.columns if c not in ['poly_idx','rel_area','poly_area', 'pix_idxs','coords']]:
        original_var = reduce(lambda s, sub: s.replace(sub, ''), SUPPORTED_STATISTICS_METHODS, var)
        original_var = original_var.replace('_', '')
            
        if (original_var not in agg_obj.ds_in.var()):
            # For auxiliary variables (from the shapefile), just copy them wholesale into the dataset
            ds_out[var] = xr.DataArray(data=agg_obj.agg[var],coords=[agg_obj.agg.poly_idx],dims=['poly_idx'])
        else:
            print(var)
            original_var = reduce(lambda s, sub: s.replace(sub, ''), SUPPORTED_STATISTICS_METHODS, var)
            original_var = original_var.replace('_', '')
            # For data variables (from the input grid), create empty array
            ds_out[var] = xr.DataArray(data=np.zeros((len(agg_obj.agg),
                                                         *[agg_obj.ds_in[original_var].sizes[k] for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]))*np.nan,
                                       dims=['poly_idx',*[k for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]],
                                       coords=[[k for k in agg_obj.agg.poly_idx],*[agg_obj.ds_in[original_var][k].values for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]])

            # Now insert aggregated values 
            for poly_idx in agg_obj.agg.poly_idx:
                ds_out[var].loc[{'poly_idx':poly_idx}] = np.squeeze(agg_obj.agg.loc[poly_idx,var])

VV_min
VV_max
VH_min
VH_max
VV_mean
VH_mean


In [35]:
ds_out

<xarray.Dataset>
Dimensions:     (poly_idx: 101, t: 12)
Coordinates:
  * poly_idx    (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * t           (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-03
Data variables: (12/23)
    Unnamed: 0  (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
    ID          (poly_idx) int64 2894152 2347633 2970168 ... 2060657 2614883
    FID         (poly_idx) object 'PRGMFR75E13F132Z-1688030' ... 'NTRSFR65C25...
    CUAA        (poly_idx) object 'PRGMFR75E13F132Z' ... 'NTRSFR65C25I431Z'
    CODE        (poly_idx) object 'AP2' 'AP2' 'AP2' 'AP2' ... 'AP2' 'AP2' 'AP2'
    DESCR_IT    (poly_idx) object 'Prato stabile' ... 'Prato stabile'
    ...          ...
    VV_min      (poly_idx, t) float64 nan nan nan nan ... 0.005941 0.0 0.005757
    VV_max      (poly_idx, t) float64 nan nan nan nan ... 0.0 0.2604 0.0 0.3252
    VH_min      (poly_idx, t) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    VH_max      (poly_idx, t) float64 nan nan nan nan ... 0.0 0.0653 0.0 0.06992
    VV_mean     (poly_idx, t) float64 nan nan nan nan ... 0.0 0.04976 0.0 0.0554
    VH_mean     (poly_idx, t) float64 nan nan nan nan ... 0.01559 0.0 0.01323

In [34]:
aggregated.to_dataset()

KeyError: 'dem'

In [10]:
weightmap.agg

,Unnamed: 0,ID,FID,CUAA,CODE,DESCR_IT,DESCR_DE,area,dem_mean,dem_median,...,slo_median,asp_mean,asp_median,north_mean,north_medi,dem_class,poly_idx,rel_area,pix_idxs,coords
0,0,2894152,PRGMFR75E13F132Z-1688030,PRGMFR75E13F132Z,AP2,Prato stabile,Wiese (Dauerwiese),56,1301.867221,1301.705200,...,17.806307,85.131834,84.665092,1.000000,1.0,1300.0,0,"[[0.022842935418290994, 0.3365725913202404, 0....","[131865, 131866, 135838, 135839, 139810, 139811]","[(46.4681533573721, 11.026701124126358), (46.4..."
1,1,2347633,RNRGTH80H18F132V-59,RNRGTH80H18F132V,AP2,Prato stabile,Wiese (Dauerwiese),31147,1302.440111,1302.553589,...,22.398680,145.304115,153.931000,0.043887,0.0,1300.0,1,"[[0.0001881757677836381, 0.0014258047322418242...","[6257564, 6257565, 6257566, 6257567, 6257568, ...","[(46.653154691735885, 10.946677849145395), (46..."
2,2,2970168,WSSCRS85L19A952F-2158250,WSSCRS85L19A952F,AP2,Prato stabile,Wiese (Dauerwiese),842,1318.083754,1317.940063,...,12.262682,228.961556,228.782425,0.000000,0.0,1300.0,2,"[[0.022418200093204426, 0.11321393053564119, 0...","[994886, 994887, 994888, 994889, 998858, 99885...","[(46.494187916099044, 11.132279058134225), (46..."
3,3,2713883,RCHRST53P06F849C-1949169,RCHRST53P06F849C,AP2,Prato stabile,Wiese (Dauerwiese),81,658.964977,660.032715,...,28.693043,140.177669,139.884628,0.000000,0.0,700.0,3,"[[0.12855827031450892, 0.019758462739474066, 0...","[6313591, 6313592, 6317564, 6317565, 6321538]","[(46.65483434068601, 10.995267693774016), (46...."
4,4,3043605,HLLNBR89A02A952X-23,HLLNBR89A02A952X,AP2,Prato stabile,Wiese (Dauerwiese),68,1278.170593,1278.170593,...,21.661054,130.486549,130.486549,0.000000,0.0,1300.0,4,"[[0.012417318903730686, 0.058952069347652576, ...","[2597433, 2601405, 2601406, 2605378, 2605379]","[(46.54253781087764, 11.30360325104699), (46.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,2609334,GGRMTN74C58F132B-984682,GGRMTN74C58F132B,AP2,Prato stabile,Wiese (Dauerwiese),1266,651.343887,651.285156,...,23.579442,112.737972,114.252159,0.000000,0.0,700.0,96,"[[0.0014607863492052632, 0.023210380457563202,...","[5845899, 5845900, 5845901, 5845902, 5845903, ...","[(46.64067729953496, 11.129879559634047), (46...."
97,97,2687416,GRBNTN67H16F118W-2052284,GRBNTN67H16F118W,AP2,Prato stabile,Wiese (Dauerwiese),2522,968.684737,968.406860,...,12.142462,252.447315,252.722717,0.000000,0.0,1000.0,97,"[[0.018254058828894727, 0.017416426248843545, ...","[3725337, 3725338, 3725339, 3725340, 3729310, ...","[(46.576610689580185, 11.252253983143163), (46..."
98,98,1963914,GSRRLF68E28F132D-2243402,GSRRLF68E28F132D,AP2,Prato stabile,Wiese (Dauerwiese),10953,1599.938267,1600.226257,...,5.682617,189.662236,198.687714,0.009091,0.0,1600.0,98,"[[0.001404113328895351, 0.004536686969915199, ...","[947455, 947456, 947457, 951427, 951428, 95142...","[(46.49274821699893, 11.161672914761414), (46...."
99,99,2060657,SPMRRT64H15A022R-2354157,SPMRRT64H15A022R,AP2,Prato stabile,Wiese (Dauerwiese),4817,636.584208,636.959778,...,10.786485,60.665624,60.499100,1.000000,1.0,600.0,99,"[[0.004048357914801504, 0.012783622476505043, ...","[2981741, 2981742, 2981743, 2981744, 2985712, ...","[(46.55417537860351, 11.174870156512398), (46...."


In [25]:
ds = s1_wgs84
wm = weightmap
# impl='for_loop'
silent=False

In [62]:
ds

<xarray.Dataset>
Dimensions:   (run: 2, loc: 4, bnds: 2)
Coordinates:
  * run       (run) int64 0 1
  * bnds      (bnds) int64 0 1
  * loc       (loc) object MultiIndex
  * lat       (loc) int64 0 0 1 1
  * lon       (loc) int64 0 1 0 1
Data variables:
    test      (run, loc) int64 0 0 2 2 1 1 3 3
    lat_bnds  (bnds, loc) float64 -0.5 -0.5 0.5 0.5 0.5 0.5 1.5 1.5
    lon_bnds  (bnds, loc) float64 -0.5 0.5 -0.5 0.5 0.5 1.5 0.5 1.5

In [32]:
b

In [ ]:
df_out = b.to_dataframe()
df_out

In [29]:
b.to_dataset()

<xarray.Dataset>
Dimensions:     (poly_idx: 101, t: 12)
Coordinates:
  * poly_idx    (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * t           (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-03
Data variables: (12/19)
    Unnamed: 0  (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
    ID          (poly_idx) int64 2894152 2347633 2970168 ... 2060657 2614883
    FID         (poly_idx) object 'PRGMFR75E13F132Z-1688030' ... 'NTRSFR65C25...
    CUAA        (poly_idx) object 'PRGMFR75E13F132Z' ... 'NTRSFR65C25I431Z'
    CODE        (poly_idx) object 'AP2' 'AP2' 'AP2' 'AP2' ... 'AP2' 'AP2' 'AP2'
    DESCR_IT    (poly_idx) object 'Prato stabile' ... 'Prato stabile'
    ...          ...
    asp_median  (poly_idx) float64 84.67 153.9 228.8 139.9 ... 198.7 60.5 158.1
    north_mean  (poly_idx) float64 1.0 0.04389 0.0 0.0 ... 0.0 0.009091 1.0 0.0
    north_medi  (poly_idx) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 1.0 0.0
    dem_class   (poly_idx) float64 1.3e+03 1.3e+03 1.3e+03 ... 600.0 1.2e+03
    VV_min      (poly_idx) object [[array([0.        , 0.00121075, 0.        ...
    VH_min      (poly_idx) object [[array([0.00000000e+00, 6.70609670e-05, 0....

In [33]:
df_ref = pd.DataFrame({'poly_idx':[0,0],'run':[0,1],'name':['test','test'],'test':[0.9999,1.9999]})

In [35]:
df_ref = df_ref.set_index(['poly_idx','run'])

In [36]:
pd.testing.assert_frame_equal(df_out,df_ref,atol=0.0001)

AssertionError: DataFrame are different

DataFrame shape mismatch
[left]:  (1212, 19)
[right]: (2, 2)

In [ ]:
pd.testing.assert_frame_equal(df_out,df_ref,atol=0.0001)

In [26]:
xa.aggregate(ds, wm, stat = "min").to_dataset()

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating VV by min...
aggregating VH by min...
all variables aggregated to polygons!


<xarray.Dataset>
Dimensions:     (poly_idx: 101, t: 12)
Coordinates:
  * poly_idx    (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * t           (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-03
Data variables: (12/19)
    Unnamed: 0  (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
    ID          (poly_idx) int64 2894152 2347633 2970168 ... 2060657 2614883
    FID         (poly_idx) object 'PRGMFR75E13F132Z-1688030' ... 'NTRSFR65C25...
    CUAA        (poly_idx) object 'PRGMFR75E13F132Z' ... 'NTRSFR65C25I431Z'
    CODE        (poly_idx) object 'AP2' 'AP2' 'AP2' 'AP2' ... 'AP2' 'AP2' 'AP2'
    DESCR_IT    (poly_idx) object 'Prato stabile' ... 'Prato stabile'
    ...          ...
    asp_median  (poly_idx) float64 84.67 153.9 228.8 139.9 ... 198.7 60.5 158.1
    north_mean  (poly_idx) float64 1.0 0.04389 0.0 0.0 ... 0.0 0.009091 1.0 0.0
    north_medi  (poly_idx) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 1.0 0.0
    dem_class   (poly_idx) float64 1.3e+03 1.3e+03 1.3e+03 ... 600.0 1.2e+03
    VV_min      (poly_idx) object [[array([0.        , 0.00121075, 0.        ...
    VH_min      (poly_idx) object [[array([0.00000000e+00, 6.70609670e-05, 0....

#### Tets

In [27]:
import pytest
import pandas as pd
import numpy as np
import xarray as xr
import shutil
import geopandas as gpd
from geopandas import testing as gpdt
from unittest import TestCase
from shapely.geometry import Polygon

In [28]:
ds = xr.Dataset({'test':(['lon','lat','run'],np.array([[[0,1],[2,3]],[[0,1],[2,3]]])),
				 'lat_bnds':(['lat','bnds'],np.array([[-0.5,0.5],[0.5,1.5]])),
				 'lon_bnds':(['lon','bnds'],np.array([[-0.5,0.5],[0.5,1.5]]))},
				coords={'lat':(['lat'],np.array([0,1])),
						'lon':(['lon'],np.array([0,1])),
						'run':(['run'],np.array([0,1])),
						'bnds':(['bnds'],np.array([0,1]))})

In [31]:
# Create polygon covering multiple pixels
gdf = {'name':['test'],
			'geometry':[Polygon([(0,0),(0,1),(1,1),(1,0),(0,0)])]}
gdf = gpd.GeoDataFrame(gdf,crs="EPSG:4326")

# Get pixel overlaps
wm = pixel_overlaps(ds,gdf)

# Get aggregate
agg = aggregate(ds,wm, stat=['mean', 'min'])

creating polygons for each pixel...
calculating overlaps between pixels and output polygons...
success!
aggregating test by mean, min...
all variables aggregated to polygons!


In [32]:
from functools import reduce


In [34]:
ds_out = agg.to_dataset()

# Build reference output dataset
ds_ref = xr.Dataset({'name':(['poly_idx'],np.array(['test']).astype(object)),
                     'test_mean':(['poly_idx','run'],np.array([[1.0,2.0]]))},
                coords={'poly_idx':(['poly_idx'],np.array([0])),
                        'run':(['run'],np.array([0,1]))})

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
#xr.testing.assert_allclose(ds_out,ds_ref,atol=0.0001)

In [23]:
ds_ref

<xarray.Dataset>
Dimensions:    (poly_idx: 1, run: 2)
Coordinates:
  * poly_idx   (poly_idx) int64 0
  * run        (run) int64 0 1
Data variables:
    name       (poly_idx) object 'test'
    test_mean  (poly_idx, run) float64 1.0 2.0

In [22]:
ds_out

<xarray.Dataset>
Dimensions:    (poly_idx: 1, run: 2)
Coordinates:
  * poly_idx   (poly_idx) int64 0
  * run        (run) int64 0 1
Data variables:
    name       (poly_idx) object 'test'
    test_mean  (poly_idx, run) float64 1.0 2.0

#### Get pixel overlaps

In [36]:
## Get pixel overlaps
ds = ds
gdf_in = gdf
weights=None
weights_target='ds'
subset_bbox = True
impl='for_loop'
silent=False

In [37]:
gdf_in = copy.deepcopy(gdf_in)

# Turn into dataset if dataarray
if type(ds)==xr.core.dataarray.DataArray:
  if ds.name is None:
    ds = ds.to_dataset(name='var')
  else:
    ds = ds.to_dataset()

# Create a polygon for each pixel
if not silent:
  print('creating polygons for each pixel...')
if subset_bbox:
    pix_agg = create_raster_polygons(ds,subset_bbox=gdf_in,weights=weights)
else:
    pix_agg = create_raster_polygons(ds,subset_bbox=None,weights=weights)

# Get overlaps between these pixel polygons and the gdf_in polygons
if not silent:
  print('calculating overlaps between pixels and output polygons...')

creating polygons for each pixel...
calculating overlaps between pixels and output polygons...


In [38]:
gdf_in['poly_idx'] = np.arange(0,len(gdf_in))

In [39]:
pix_agg['gdf_pixels'] = pix_agg['gdf_pixels'].to_crs(gdf_in.crs)

In [40]:
if np.all(gdf_in.total_bounds[[1,3]]>0):
    # If min/max lat are both in NH, use North grid
    #epsg_set = {'init':'EPSG:6931'} (change to below bc of depreciation of {'init':...} format in geopandas)
    epsg_set = 'EPSG:6931'
elif np.all(gdf_in.total_bounds[[1,3]]<0):
    # If min/max lat are both in SH, use South grid
    #epsg_set = {'init':'EPSG:6932'}
    epsg_set = 'EPSG:6932'
else:
    # Otherwise, use the global/temperate grid
    #epsg_set = {'init':'EPSG:6933'}
    epsg_set = 'EPSG:6933'

# Get GeoDataFrame of the overlaps between every pixel and the polygons
with warnings.catch_warnings():
    # Filter UserWarnings that flag when overlapping would result in 
    # lines, or point overlaps (but we only care about 3D overlaps, so
    # keep_geom_type=True is the right call, and we don't need the 
    # warning)
    warnings.filterwarnings('ignore',category=UserWarning)
    overlaps = gpd.overlay(gdf_in.to_crs(epsg_set),
                           pix_agg['gdf_pixels'].to_crs(epsg_set),
                           how='intersection')

In [41]:
ov_groups = overlaps.groupby('poly_idx')
# Calculate relative area of each overlap (so how much of the total 
# area of each polygon is taken up by each pixel), the pixels 
# corresponding to those areas, and the lat/lon coordinates of 
# those pixels
overlap_info = ov_groups.agg(rel_area=pd.NamedAgg(column='geometry',aggfunc=lambda ds: [normalize(ds.area)]),
                             poly_area=pd.NamedAgg(column='geometry',aggfunc=lambda ds: [ds.area]),
                             pix_idxs=pd.NamedAgg(column='pix_idx',aggfunc=lambda ds: [idx for idx in ds]),
                             lat=pd.NamedAgg(column='lat',aggfunc=lambda ds: [x for x in ds]),
                             lon=pd.NamedAgg(column='lon',aggfunc=lambda ds: [x for x in ds]))

In [42]:
overlap_info

,rel_area,poly_area,pix_idxs,lat,lon
poly_idx,,,,,
0,"[[0.25000926471575663, 0.25000926471575663, 0....","[[3077230007.91319, 3077230007.91319, 30770019...","[0, 1, 2, 3]","[0, 0, 1, 1]","[0, 1, 0, 1]"


In [43]:
overlap_info['coords'] = overlap_info.apply(lambda row: list(zip(row['lat'],row['lon'])),axis=1)
overlap_info = overlap_info.drop(columns=['lat','lon'])

# Reset index to make poly_idx a column for merging with gdf_in
overlap_info = overlap_info.reset_index()

In [44]:
gdf_in = pd.merge(gdf_in,overlap_info,'outer')

# Drop 'geometry' eventually, just for size/clarity
wm_out = weightmap(agg=gdf_in.drop('geometry',axis=1),
                   source_grid=pix_agg['source_grid'],
                   geometry=gdf_in.geometry)

In [45]:
if 'weights' in pix_agg['gdf_pixels'].columns:
    wm_out.weights = pix_agg['gdf_pixels'].weights

#### Aggregate

In [46]:
ds = ds
wm = wm_out

impl='for_loop'
stat=['mean']
skipna=True
interpolate_NaN=False
silent=False

In [47]:
def check_supported_statistic(stats_list):
    for stat in stats_list:
        assert stat in SUPPORTED_STATISTICS_METHODS, f"You selected {stat}. This statistic is not available. Please choose one of the following: {SUPPORTED_STATISTICS_METHODS}."

In [48]:
check_supported_statistic(stat)

In [49]:

# Make sure pixel_overlaps was correctly run if using dot product
if (impl=='dot_product') and (wm.overlap_da is None):
    raise ValueError("no 'overlap_da' was found in the `wm` input - since you're using the dot product implementation, "+
                     "make sure to run `pixel_overlaps()` with `impl='dot_product'` to avoid this error.")

# Turn into dataset if dataarray
if type(ds)==xr.core.dataarray.DataArray:
    if ds.name is None:
        warnings.warn('An unnamed xr.DataArray was inputted instead of a xr.Dataset; the output variable will be "var"')
        ds = ds.to_dataset(name='var')
    else:
        ds = ds.to_dataset()

In [50]:
ds = fix_ds(ds)

# interpolate NaNs
if interpolate_NaN == True:
    ds = ds.interpolate_na()  

# Stack 
ds = ds.stack(loc=('lat','lon'))


In [51]:
ds

<xarray.Dataset>
Dimensions:   (run: 2, loc: 4, bnds: 2)
Coordinates:
  * run       (run) int64 0 1
  * bnds      (bnds) int64 0 1
  * loc       (loc) object MultiIndex
  * lat       (loc) int64 0 0 1 1
  * lon       (loc) int64 0 1 0 1
Data variables:
    test      (run, loc) int64 0 0 2 2 1 1 3 3
    lat_bnds  (bnds, loc) float64 -0.5 -0.5 0.5 0.5 0.5 0.5 1.5 1.5
    lon_bnds  (bnds, loc) float64 -0.5 0.5 -0.5 0.5 0.5 1.5 0.5 1.5

In [52]:
# Adjust grid of [ds] if necessary to match 
ds = subset_find(ds,wm.source_grid)

# Set weights; or replace with ones if no additional weight information
#if wm.weights != 'nowghts':
if type(wm.weights) == pd.core.series.Series:
    weights = np.array([float(k) for k in wm.weights])
else:
    if wm.weights != 'nowghts':
        warnings.warn('wm.weights is: \n '+print(wm.weights)+
                        ', \n which is not a supported weight vector (in a pandas series) '+
                        'or "nowghts" as a string. Assuming no weights are included...')
    if impl=='dot_product':
        weights = np.ones((len(wm.overlap_da['loc'])))
    elif impl=='for_loop':
        weights = np.ones((len(wm.source_grid['lat'])))

In [383]:
wm.agg

,name,poly_idx,rel_area,pix_idxs,coords
0,test,0,"[[0.25000926471575663, 0.25000926471575663, 0....","[0, 1, 2, 3]","[(0, 0), (0, 1), (1, 0), (1, 1)]"


In [384]:

for var in ds:
    # Process for every variable that has locational information, but isn't a 
    # bound variable
    if ('bnds' not in ds[var].dims) & ('loc' in ds[var].dims):
        if not silent:
            print('aggregating '+var+'...')
        # Create the column for the relevant variable
        wm.agg[var] = None

        # Get weighted average of variable based on pixel overlap + other weights
        for poly_idx in wm.agg.poly_idx:
            # Get average value of variable over the polygon; weighted by 
            # how much of the pixel area is in the polygon, and by (optionally)
            # a separate gridded weight. This if statement checks 
            # - if the weights output for this polygon is just "np.nan", which 
            #   indicates there's no overlap between the pixel grid and polygons
            # - [this has been pushed down a few lines] or, if all the pixels in 
            #   the grid have just nan values for this variable
            # in both cases; the "aggregated variable" is just a vector of nans. 
            if not np.isnan(wm.agg.iloc[poly_idx,:].pix_idxs).all():
                # Get the dimensions of the variable that aren't "loc" (location)
                other_dims = [k for k in np.atleast_1d(ds[var].dims) if k != 'loc']
                # Check first if any nans are "complete" (meaning that a pixel 
                # either has values for each step, or nans for each step - if
                # there are random nans within a pixel, throw a warning)
                if not xr.Dataset.equals(np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).any(other_dims),
                                          np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(other_dims)):
                    warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
                                   ', '.join([k for k in ds[var].dims if k != 'loc'])+
                                   '. The code can currently only deal with pixels for which the '+
                                   '*entire* pixel has nan values in all dimensions, however there is currently no '+
                                   ' support for data in which pixels have only some nan values. The aggregation '+
                                   'calculation is likely incorrect.')

                if not np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(): 
                    # Get relative areas for the pixels overlapping with this Polygon
                    tmp_areas = np.atleast_1d(np.squeeze(wm.agg.iloc[poly_idx,:].rel_area))
                    # Replace overlapping pixel areas with nans if the corresponding pixel
                    # is only composed of nans
                    tmp_areas[np.array(np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(other_dims).values)] = np.nan
                    # Calculate the normalized area+weight of each pixel (taking into account
                    # nans)
                    normed_areaweights = normalize(tmp_areas*weights[wm.agg.iloc[poly_idx,:].pix_idxs],drop_na=True)

                    # Take the weighted average of all the pixel values to calculate the 
                    # aggregated value for the shapefile
                    wm.agg.loc[poly_idx,var] = [[((ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)*
                                                     normed_areaweights).
                                                    sum('loc')).values]]

                else:
                    wm.agg.loc[poly_idx,var] = [[(ds[var].isel(loc=0)*np.nan).values]]

            else:
                wm.agg.loc[poly_idx,var] = [[(ds[var].isel(loc=0)*np.nan).values]]

# Put in class format
agg_out = aggregated(agg=wm.agg,source_grid=wm.source_grid,
                     geometry=wm.geometry,ds_in=ds,weights=wm.weights)

aggregating test...


In [395]:
wm.agg

,name,poly_idx,rel_area,pix_idxs,coords,test
0,test,0,"[[0.25000926471575663, 0.25000926471575663, 0....","[0, 1, 2, 3]","[(0, 0), (0, 1), (1, 0), (1, 1)]","[[[0.9999629411369734, 1.9999629411369735]]]"


In [53]:
for var in ds:
    # Process for every variable that has locational information, but isn't a 
    # bound variable
    if ('bnds' not in ds[var].dims) & ('loc' in ds[var].dims):
        if not silent:
            all_stat = ', '.join(stat)
            print('aggregating '+var+' by '+all_stat+ '...')

        # Get weighted average of variable based on pixel overlap + other weights
        for poly_idx in wm.agg.poly_idx:
            # Get average value of variable over the polygon; weighted by 
            # how much of the pixel area is in the polygon, and by (optionally)
            # a separate gridded weight. This if statement checks 
            # - if the weights output for this polygon is just "np.nan", which 
            #   indicates there's no overlap between the pixel grid and polygons
            # - [this has been pushed down a few lines] or, if all the pixels in 
            #   the grid have just nan values for this variable
            # in both cases; the "aggregated variable" is just a vector of nans. 
            if not np.isnan(wm.agg.iloc[poly_idx,:].pix_idxs).all():
                # Get the dimensions of the variable that aren't "loc" (location)
                other_dims = [k for k in np.atleast_1d(ds[var].dims) if k != 'loc']
                # Check first if any nans are "complete" (meaning that a pixel 
                # either has values for each step, or nans for each step - if
                # there are random nans within a pixel, throw a warning)
                if not xr.Dataset.equals(np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).any(other_dims),
                                          np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(other_dims)):
                    warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
                                   ', '.join([k for k in ds[var].dims if k != 'loc'])+
                                   '. The code can currently only deal with pixels for which the '+
                                   '*entire* pixel has nan values in all dimensions, however there is currently no '+
                                   ' support for data in which pixels have only some nan values. The aggregation '+
                                   'calculation is likely incorrect.')

                if not np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(): 
                    # Get relative areas for the pixels overlapping with this Polygon
                    tmp_areas = np.atleast_1d(np.squeeze(wm.agg.iloc[poly_idx,:].rel_area))
                    # Replace overlapping pixel areas with nans if the corresponding pixel
                    # is only composed of nans
                    tmp_areas[np.array(np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(other_dims).values)] = np.nan
                    # Calculate the normalized area+weight of each pixel (taking into account
                    # nans)
                    
                    # Weights for weighted mean 
                    normed_areaweights = normalize(tmp_areas*weights[wm.agg.iloc[poly_idx,:].pix_idxs],drop_na=True)
                    
                    # Weights for weighted sum
                    areaweights = tmp_areas*weights[wm.agg.iloc[poly_idx,:].pix_idxs]

                    # Take the values of all pixel values in the current polygon
                    pix_in_poly = ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)
                    pixel_area = np.atleast_1d(np.squeeze(wm.agg.iloc[poly_idx,:].poly_area))

                    # Apply aggregation statistic method to the weighted pixel values
                    for i in stat: 
                        if i == "mean": # weighted average of all the pixel values
                            stat_i = "mean"
                            var_stat = var+'_'+stat_i
                            wm.agg[var_stat] = None
                            wm.agg.loc[poly_idx,var_stat] = [[(pix_in_poly*normed_areaweights).sum('loc', skipna=skipna).values]]
                            continue
                        elif i == "max":
                            stat_i = "max"
                            var_stat = var+'_'+stat_i
                            wm.agg[var_stat] = None
                            wm.agg.loc[poly_idx,var_stat] = [[pix_in_poly.max('loc', skipna=skipna)]]
                            continue
                        elif i == "min":
                            stat_i = "min"
                            var_stat = var+'_'+stat_i
                            wm.agg[var_stat] = None
                            wm.agg.loc[poly_idx,var_stat] = [[pix_in_poly.min('loc', skipna=skipna)]]
                            continue
                        elif i == "sum": # mean*count or the same: pixel value * percentage of pixel area covered by polygon
                            stat_i = "sum"
                            var_stat = var+'_'+stat_i
                            #wm.agg[var_stat] = None
                            # wm.agg.loc[poly_idx,var_stat] = [[(pix_in_poly*normed_areaweights).sum('loc', skipna=skipna).values *
                            #                                   list(map(sum, list(map(sum, wm.agg.rel_area.values))))[0] / list(map(max, list(map(max, wm.agg.rel_area.values))))[0]]]
                            wm.agg.loc[poly_idx,var_stat] = [[(pix_in_poly*(pixel_area/pixel_area.max())).sum('loc', skipna=skipna).values]]
                            continue
                        elif i == "median":
                            stat_i = "median"
                            var_stat = var+'_'+stat_i
                            wm.agg[var_stat] = None
                            wm.agg.loc[poly_idx,var_stat] = [[pix_in_poly.median('loc', skipna=skipna)]]
                            continue
                        elif i == "count": # polygon area divided by pixel size gives exact count
                            stat_i = "count"
                            var_stat = var+'_'+stat_i
                            wm.agg[var_stat] = None
                            # wm.agg.loc[poly_idx,var_stat] = [[list(map(sum, list(map(sum, wm.agg.rel_area.values))))[0] / list(map(max, list(map(max, wm.agg.rel_area.values))))[0]]]
                            wm.agg.loc[poly_idx,var_stat] = [[pixel_area.sum()/pixel_area.max()]]
                            continue
                        elif "std" in stat:
                            stat_i = "std"
                            var_stat = var+'_'+stat_i
                            wm.agg[var_stat] = None                               
                            wm.agg.loc[poly_idx,var_stat] = [[(pix_in_poly*weights[wm.agg.iloc[poly_idx,:].pix_idxs]).std('loc', skipna=skipna)]]

                else:
                    wm.agg.loc[poly_idx,var] = [[(ds[var].isel(loc=0)*np.nan).values]]

            else:
                wm.agg.loc[poly_idx,var] = [[(ds[var].isel(loc=0)*np.nan).values]]

# Put in class format
agg_out = aggregated(agg=wm.agg,source_grid=wm.source_grid,
                     geometry=wm.geometry,ds_in=ds,weights=wm.weights)

aggregating test by mean...


In [26]:
pix_in_poly

NameError: name 'pix_in_poly' is not defined

In [ ]:
pix_in_poly

In [441]:
wm.agg

,name,poly_idx,rel_area,poly_area,pix_idxs,coords,test_mean
0,test,0,"[[0.25000926471575663, 0.25000926471575663, 0....","[[3077230007.91319, 3077230007.91319, 30770019...","[0, 1, 2, 3]","[(0, 0), (0, 1), (1, 0), (1, 1)]","[[[0.9999629411369734, 1.9999629411369735]]]"


In [385]:
ds_out2 = agg.to_dataset()
ds_out2

<xarray.Dataset>
Dimensions:   (poly_idx: 1, run: 2)
Coordinates:
  * poly_idx  (poly_idx) int64 0
  * run       (run) int64 0 1
Data variables:
    name      (poly_idx) object 'test'

#### Export Part

In [442]:
agg_obj=agg_out
loc_dim='poly_idx'

In [443]:
ds_out = xr.Dataset(coords={'poly_idx':(['poly_idx'],agg_obj.agg.poly_idx.values)})

In [444]:
ds_out

<xarray.Dataset>
Dimensions:   (poly_idx: 1)
Coordinates:
  * poly_idx  (poly_idx) int64 0
Data variables:
    *empty*

In [445]:
[c for c in agg_obj.agg.columns]

['name',
 'poly_idx',
 'rel_area',
 'poly_area',
 'pix_idxs',
 'coords',
 'test_mean']

In [454]:
agg_obj.ds_in.var()

<xarray.Dataset>
Dimensions:   ()
Data variables:
    test      float64 1.25
    lat_bnds  float64 0.5
    lon_bnds  float64 0.5

In [457]:
var

'test_mean'

In [458]:
from functools import reduce


In [461]:
new_string = reduce(lambda s, sub: s.replace(sub, ''), SUPPORTED_STATISTICS_METHODS, var)
new_string = new_string.replace('_', '')
new_string

'test'

In [463]:
for var in [c for c in agg_obj.agg.columns if c not in ['poly_idx','rel_area','poly_area', 'pix_idxs','coords']]:    
    if (var not in agg_obj.ds_in.var()) and (not any(var_stat in var for var_stat in SUPPORTED_STATISTICS_METHODS) ):
        
        # For auxiliary variables (from the shapefile), just copy them wholesale into the dataset
        ds_out[var] = xr.DataArray(data=agg_obj.agg[var],coords=[agg_obj.agg.poly_idx],dims=['poly_idx'])
    else:
        original_var = reduce(lambda s, sub: s.replace(sub, ''), SUPPORTED_STATISTICS_METHODS, var)
        original_var = original_var.replace('_', '')
        # For data variables (from the input grid), create empty array
        ds_out[var] = xr.DataArray(data=np.zeros((len(agg_obj.agg),
                                                     *[agg_obj.ds_in[original_var].sizes[k] for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]))*np.nan,
                                   dims=['poly_idx',*[k for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]],
                                   coords=[[k for k in agg_obj.agg.poly_idx],*[agg_obj.ds_in[original_var][k].values for k in agg_obj.ds_in[original_var].sizes.keys() if k not in ['lat','lon','loc']]])

        # Now insert aggregated values 
        for poly_idx in agg_obj.agg.poly_idx:
            ds_out[var].loc[{'poly_idx':poly_idx}] = np.squeeze(agg_obj.agg.loc[poly_idx,var])

In [464]:
ds_out

<xarray.Dataset>
Dimensions:    (poly_idx: 1, run: 2)
Coordinates:
  * poly_idx   (poly_idx) int64 0
  * run        (run) int64 0 1
Data variables:
    name       (poly_idx) object 'test'
    test_mean  (poly_idx, run) float64 1.0 2.0

In [391]:
ds_out

<xarray.Dataset>
Dimensions:   (poly_idx: 1, run: 2)
Coordinates:
  * poly_idx  (poly_idx) int64 0
  * run       (run) int64 0 1
Data variables:
    name      (poly_idx) object 'test'
    test      (poly_idx, run) float64 1.0 2.0

In [295]:
ds_out

<xarray.Dataset>
Dimensions:    (poly_idx: 1)
Coordinates:
  * poly_idx   (poly_idx) int64 0
Data variables:
    name       (poly_idx) object 'test'
    test_mean  (poly_idx) object [[array([0.99996294, 1.99996294])]]

In [465]:
# Add non-geographic coordinates for the variables to be aggregated
for crd in [k for k in agg_obj.ds_in.sizes.keys() if (k not in ['lat','lon','loc','bnds'])]:
    ds_out[crd] = xr.DataArray(dims=[crd],
                               data=agg_obj.ds_in[crd].values,
                               coords=[agg_obj.ds_in[crd].values])

In [297]:
ds_out

<xarray.Dataset>
Dimensions:    (poly_idx: 1, run: 2)
Coordinates:
  * poly_idx   (poly_idx) int64 0
  * run        (run) int64 0 1
Data variables:
    name       (poly_idx) object 'test'
    test_mean  (poly_idx) object [[array([0.99996294, 1.99996294])]]

In [466]:
# Rename poly_idx if desired
if loc_dim != 'poly_idx':
    ds_out = ds_out.rename({'poly_idx':loc_dim})

In [24]:
ds_ref = xr.Dataset({'name':(['poly_idx'],np.array(['test']).astype(object)),
                     'test_mean':(['poly_idx','run'],np.array([[1.0,2.0]]))},
                coords={'poly_idx':(['poly_idx'],np.array([0])),
                        'run':(['run'],np.array([0,1]))})

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
xr.testing.assert_allclose(ds_out,ds_ref,atol=0.0001)

In [25]:
df_out = agg.to_dataframe()

# Build reference output dataframe
df_ref = pd.DataFrame({'poly_idx':[0,0],'run':[0,1],'name':['test','test'],'test_mean':[0.9999,1.9999]})
df_ref = df_ref.set_index(['poly_idx','run'])

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
pd.testing.assert_frame_equal(df_out,df_ref,atol=0.0001)

### Export CSV

In [496]:
csv_out = agg_obj.agg.drop(columns=['rel_area','poly_area','pix_idxs','coords','poly_idx'])

In [497]:
csv_out

,name,test_mean
0,test,"[[[0.9999629411369734, 1.9999629411369735]]]"


In [498]:
add_geom=False

In [499]:
agg_obj.ds_in

<xarray.Dataset>
Dimensions:   (run: 2, loc: 4, bnds: 2)
Coordinates:
  * run       (run) int64 0 1
  * bnds      (bnds) int64 0 1
  * loc       (loc) object MultiIndex
  * lat       (loc) int64 0 0 1 1
  * lon       (loc) int64 0 1 0 1
Data variables:
    test      (run, loc) int64 0 0 2 2 1 1 3 3
    lat_bnds  (bnds, loc) float64 -0.5 -0.5 0.5 0.5 0.5 0.5 1.5 1.5
    lon_bnds  (bnds, loc) float64 -0.5 0.5 -0.5 0.5 0.5 1.5 0.5 1.5

In [500]:
[c for c in agg_obj.agg.columns if ((c not in ['poly_idx','rel_area','poly_area','pix_idxs','coords']) )]

['name', 'test_mean']

In [501]:
# Now expand the aggregated variable into multiple columns
for var in [c for c in agg_obj.agg.columns if ((c not in ['poly_idx','rel_area','poly_area','pix_idxs','coords']) )]:
    print(var)
    original_var = reduce(lambda s, sub: s.replace(sub, ''), SUPPORTED_STATISTICS_METHODS, var)
    original_var = original_var.replace('_', '')
    if original_var in agg_obj.ds_in.var():
        print(original_var)
        # NOT YET IMPLEMENTED: dynamic column naming - so if it recognizes 
        # it as a date, then instead of doing var0, var1, var2,... it does
        # varYYYYMMDD etc.
        # These are the coordinates of the variable in the original raster
        dimsteps = [agg_obj.ds_in[original_var][d].values for d in agg_obj.ds_in[original_var].sizes.keys() if d not in ['lat','lon','loc']]
        # ALSO SHOULD check to see if the variables are multi-D - if they are
        # there are two options: 
        # - a multi-index column title (var0-0, var0-1)
        # - or an error saying csv output is not supported for this

        # Reshape the variable wide and name the columns [var]0, [var]1,...
        if len(dimsteps) == 0:
            # (in this case, all it does is move from one list per row to 
            # one value per row)
            expanded_var = (pd.DataFrame(pd.DataFrame(csv_out[var].to_list())[0].to_list(),
                                     columns=[var]))
        else:
            expanded_var = (pd.DataFrame(pd.DataFrame(csv_out[var].to_list())[0].to_list(),
                                     columns=[var+str(idx) for idx in np.arange(0,len(csv_out[var][0][0]))]))
        # Append to existing series
        csv_out = pd.concat([csv_out.drop(columns=(var)),
                             expanded_var],
                            axis=1)
        del expanded_var

if add_geom:
    # Return the geometry from the original geopandas.GeoDataFrame
    csv_out.geometry = agg_obj.geometry

name
test_mean
test


In [505]:
ds_out

<xarray.Dataset>
Dimensions:    (poly_idx: 1, run: 2)
Coordinates:
  * poly_idx   (poly_idx) int64 0
  * run        (run) int64 0 1
Data variables:
    name       (poly_idx) object 'test'
    test_mean  (poly_idx, run) float64 1.0 2.0

In [509]:
#df_out = agg.to_dataframe()

#df_out = ds_out.to_dataframe()
# Build reference output dataframe
df_ref = pd.DataFrame({'poly_idx':[0,0],'run':[0,1],'name':['test','test'],'test_mean':[0.9999,1.9999]})
df_ref = df_ref.set_index(['poly_idx','run'])

# Assert equal within tolerance, again likely due to very slight 
# variation off from actual 1.0, 2.0 due to crs
pd.testing.assert_frame_equal(df_out,df_ref,atol=0.0001)

In [508]:
df_out = ds_out.copy()

In [507]:
ds_out

name  test_mean
poly_idx run                 
0        0    test   0.999963
         1    test   1.999963

In [503]:
df_out

name
poly_idx run      
0        0    test
         1    test

In [504]:
df_ref

name  test_mean
poly_idx run                 
0        0    test     0.9999
         1    test     1.9999

In [480]:
df_ref

name    test
poly_idx run              
0        0    test  0.9999
         1    test  1.9999

In [484]:
df_ref

name  test_mean
poly_idx run                 
0        0    test     0.9999
         1    test     1.9999